# Imports des librairies et transformation des fichiers csv en dataframes

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd  
import numpy as np

In [ ]:
logs_err = pd.read_csv('../Data/241016_LogETLError.csv', sep=';', dtype={'Program_Id': str, 'Schedules_Id': str, 'Schedules_Name': str})
logs = pd.read_csv('../Data/241016_LogETL.csv',sep=';')

# Visualisation des logs avec ERREUR

### Une première vue avec un scatter graph, pas la visualisation la plus adaptée

J'ajoute des filtres pour abaisser l'échelle des ordonnées. En effet, le 2020/08/29 il y a eu 197k erreurs. Le 2024/10/03 il y en a 4k. 

In [ ]:
# Filtrer pour enlever le point du 2024-08-29 et 2024-10-03 pour Program_Name "ETL sur ordre"
logs_err = logs_err[~((logs_err['Date'] == '2024-08-29') & (logs_err['Program_Name'] == 'ETL sur ordre'))]
logs_err = logs_err[~((logs_err['Date'] == '2024-10-03') & (logs_err['Program_Name'] == 'ETL sur ordre'))]

# Compter le nombre d'occurrences de chaque Program_Name par date
counts = logs_err.groupby(['Date', 'Program_Name'])['Date'].count().reset_index(name='Count')

# Merger avec dfDate


# Tracer le nombre de Program_Name en fonction du temps
plt.figure(figsize=(12, 6))
for program_name in counts['Program_Name'].unique():
    subset = counts[counts['Program_Name'] == program_name]
    plt.plot(subset['Date'], subset['Count'], marker='o', label=program_name)

plt.title("Nombre d'erreurs par Program Name en fonction du temps")
plt.xlabel("Date")
plt.ylabel("Nombre d'occurrences")
plt.xticks(rotation=45)  # Pour que les dates soient lisibles
plt.legend(title='Program Name')
plt.grid(True)
plt.tight_layout()  # Ajuste les marges
plt.show()

### Une deuxième vue avec un bar graph, plus appropriée

In [ ]:
# Filtrer pour enlever le point du 2024-08-29 et 2024-10-03 pour Program_Name "ETL sur ordre"
logs_err = logs_err[~((logs_err['Date'] == '2024-08-29') & (logs_err['Program_Name'] == 'ETL sur ordre'))]
logs_err = logs_err[~((logs_err['Date'] == '2024-10-03') & (logs_err['Program_Name'] == 'ETL sur ordre'))]

# Compter le nombre d'occurrences de chaque Program_Name par date
counts = logs_err.groupby(['Date', 'Program_Name'])['Date'].count().reset_index(name='Count')

# Créer un tableau pivot pour faciliter le traçage
pivot_counts = counts.pivot(index='Date', columns='Program_Name', values='Count').fillna(0)

# Tracer le nombre d'erreurs par Program_Name en fonction du temps avec des barres
plt.figure(figsize=(12, 6))
pivot_counts.plot(kind='bar', width=2.8, ax=plt.gca())

# Réglages des axes
plt.title("Nombre d'erreurs par Program Name en fonction du temps")
plt.xlabel("Date")
plt.ylabel("Nombre d'occurrences")
plt.xticks(rotation=45)  # Pour que les dates soient lisibles
plt.legend(title='Program Name')
plt.grid(axis='y')
plt.tight_layout()  # Ajuste les marges
plt.show()

Remaque : Il manque les dates pour lesquelles il n'y a pas eu d'erreur. Il faudrait merge avec un vecteur de toutes les dates possibles... 

### Dans le détail, il y a seulement 610 logs erreurs si on ignore le 29/08/2024 et le 03/01/2024

In [ ]:
# pour avoir le détail du nombre de lignes dans les logs erreur qui ne concernent pas le 29/08/2024 ou le 
print (len(logs_err['Date']))
logs_err['Date'].value_counts()

### On regarde le détail pour un ETL

In [ ]:
#Filtrer sur un ETL
count_ETL=counts[(counts['Program_Name']=='ETL BEXT')]

# Tracer un bar graph
plt.figure(figsize=(12, 6))
plt.bar(count_ETL['Date'], count_ETL['Count'], color='skyblue')

# Ajouter des labels et un titre
plt.title("Occurrences des logs erreurs du programme 'ETL BEXT' par date")
plt.xlabel("Date")
plt.ylabel("Nombre d'occurrences")
plt.xticks(rotation=45)  # Pour rendre les dates lisibles

plt.tight_layout()  # Ajuste les marges pour une meilleure lisibilité
plt.show()

# Visualisation des logs

### Une première vue avec un scatter graph, pas la visualisation la plus adaptée

In [ ]:
import matplotlib.pyplot as plt

# Enlève les lignes où Date est NaN
logs = logs.dropna(subset=['Date']) 

# Compter le nombre d'occurrences de chaque Program_Name par date
counts = logs.groupby(['Date', 'Program_Name'])['Date'].count().reset_index(name='Count')

# Tracer le nombre de Program_Name en fonction du temps
plt.figure(figsize=(12, 6))
for program_name in counts['Program_Name'].unique():
    subset = counts[counts['Program_Name'] == program_name]
    plt.plot(subset['Date'], subset['Count'], marker='', label=program_name)

plt.title("Nombre d'execution sans erreur par Program Name en fonction du temps")
plt.xlabel("Date")
plt.ylabel("Nombre d'occurrences")
plt.xticks(rotation=45)  # Pour que les dates soient lisibles
plt.legend(title='Program Name')
plt.grid(False)
#plt.tight_layout()  # Ajuste les marges
plt.show()

### On regarde le détail pour un ETL

In [ ]:
#Filtrer sur ETL
count_ETL=counts[(counts['Program_Name']=='ETL Achats')]

# Tracer un bar graph
plt.figure(figsize=(12, 6))
plt.bar(count_ETL['Date'], count_ETL['Count'], color='skyblue')

# Ajouter des labels et un titre
plt.title("Occurrences des logs du programme 'ETL Achats' par date")
plt.xlabel("Date")
plt.ylabel("Nombre d'occurrences")
plt.xticks(rotation=45)  # Pour rendre les dates lisibles

plt.tight_layout()  # Ajuste les marges pour une meilleure lisibilité
plt.show()

# Problème : on voit rien si on regarde tous les ETL ensemble. => On va les catégoriser par fréquence d'utilisation

Pour catégoriser, on va classer les Program_Name selon la moyenne quotidienne de leur count => ça revient à calculer la moyenne du nb d'execution quotidienne de chaque Program_Name

Pour calculer cette moyenne, il faut refaire une table count qui prend 0 quand il n'y a pas de log pour un jour donné. On va donc faire un merge avec dfDateprog, une table qui prend toutes les combianisons de Date et Program Name possibles. 

In [ ]:
# Extraire les dates et les program names et les mettre dans des vecteurs (dans des df)
dfDate=pd.DataFrame({'Date':logs['Date'].unique()})
dfProgramName=pd.DataFrame({'Program_Name':logs['Program_Name'].unique()})

# Créer toutes les combinaisons possibles de Date et Program_Name => produit cartésien
dfDateProg = dfDate.merge(dfProgramName, how='cross')
print(dfDateProg)

In [ ]:
print(counts.sort_values(by='Count', ascending=False))

In [ ]:
# Fusion de count et dfDateProg
adjustedCount=dfDateProg.merge(counts, on=['Date','Program_Name'], how='left')

#remplacer les valeurs NaN par 0
adjustedCount = adjustedCount.fillna(0)

print(adjustedCount)

In [ ]:
# Calculer la moyenne des Count par Program_Name
mean_counts = adjustedCount.groupby('Program_Name')['Count'].mean().reset_index(name='Mean_Count')
max_counts = adjustedCount.groupby('Program_Name')['Count'].max().reset_index(name='Max_Count')
min_counts = adjustedCount.groupby('Program_Name')['Count'].min().reset_index(name='Min_Count')
sum_counts = adjustedCount.groupby('Program_Name')['Count'].sum().reset_index(name='Sum_Count')

# Fusionner les DataFrames sur 'Program_Name'
stats_counts = mean_counts.merge(max_counts, on='Program_Name').merge(min_counts, on='Program_Name').merge(sum_counts, on='Program_Name')

# Trier par Mean_Count en ordre décroissant
stats_counts = stats_counts.sort_values(by='Mean_Count', ascending=False)

# Afficher le DataFrame mis à jour
print(stats_counts)

On peut catégoriser par la moyenne. 